<a href="https://colab.research.google.com/github/jacobdwatters/Machine-Learning-Basics/blob/main/01_logistic_regression_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class LogisticRegression:
  def __init__(self):
    self.w = np.array([0.98, 1.43]) # TEMP
    self.lr=0.01

  def sigmoid(self, X):
    return 1 / (1+np.exp(-X@self.w))

  def sigmoid_derivative(self, X):
    sig = self.sigmoid(X)
    return (sig*(1-sig))@X

  def bce(X):
    pass

  def fit(self, X, y):
    pass

X = np.array([[1, -1], 
              [1, 2], 
              [1, 3], 
              [1, 4], 
              [1, 5]])
logreg = LogisticRegression()
print(logreg.sigmoid_derivative(X))

[ 0.26497095 -0.17490491]
